# Chess using Random & MinMax AI

By **Christa Baca (@cjb-dev)**







## Description

The goal of this program is to create two AIs that will play against each other in a chess match: **Random AI (black)** and **MinMax AI (white)**. This was all done without using any premade chess libraries.

This notebook was originally created as part of an assignment for my Artificial Intelligence class. I have since made some minor modifications to update formatting, add helper print statements for easier viewership/explanation, and change the display to show unicode characters for each chess piece (since ♕ looks much nicer than 'Q').


## Part 1: Import Libraries

The code here imports the libraries necessary for this program. I did not add any other libraries that weren't already in the skeleton code for the assignment.

In [1]:
# Add only your imports here - following are the ones that may be enough to finish the assignment
import random
import time
from IPython.display import clear_output
import sys


## Part 2: Create Chess Board

The main use of this code block is to write functions to initialize the board, draw the board, get the board state, and move the chess piece for the current turn.


In [2]:
def ChessBoardSetup():
    # initialize and return a chess board - create a 2D 8x8 array that has the value for each cell
    # USE the following characters for the chess pieces - lower-case for BLACK and upper-case for WHITE
    # . for empty board cell
    # p/P for pawn
    # r/R for rook
    # t/T for knight
    # b/B for bishop
    # q/Q for queen
    # k/K for king
    board = [[],[],[],[],[],[],[],[]]
    board[0] = ['r','t','b','q','k','b','t','r']
    board[1] = ['p','p','p','p','p','p','p','p']
    board[2] = ['.','.','.','.','.','.','.','.']
    board[3] = ['.','.','.','.','.','.','.','.']
    board[4] = ['.','.','.','.','.','.','.','.']
    board[5] = ['.','.','.','.','.','.','.','.']
    board[6] = ['P','P','P','P','P','P','P','P']
    board[7] = ['R','T','B','Q','K','B','T','R']
    
    return board

# Function to print the board
def DrawBoard(board):
    # Example of board output before converting characters to unicode:
    # r t b q k b t r
    # p p p p p p p p
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # P P P P P P P P
    # R T B Q K B T R

    # dictionary to hold unicode symbols for each chess piece
    unicode_pieces = {
      'P': u'♟', 
      'T': u'♞', 
      'R': u'♜', 
      'B': u'♝', 
      'Q': u'♛',
      'K': u'♚', 
      'p': u'♙',
      't': u'♘',
      'r': u'♖', 
      'b': u'♗', 
      'q': u'♕', 
      'k': u'♔', 
      '.': '\t'.expandtabs(2)
    }

    # Iterates through board rows
    for row in board:
      # Iterates through chess pieces in each row
      for chess_piece in row:
        # Prints chess piece
        print(unicode_pieces[chess_piece], end='')
      # Adds new line before printing chess pieces in next row
      print()


def MovePiece(board, move):
    # write code to move the one chess piece
    # you do not have to worry about the validity of the move - this will be done before calling this function
    # this function will at least take the move (from-piece and to-piece) as input and return the new board layout
    
    # Find to-piece and from-piece from given move positions on the board
    fromPiece = board[move[0][0]][move[0][1]]
    toPiece = board[move[1][0]][move[1][1]]

    # Move from-piece to to-piece to change the board
    board[move[1][0]][move[1][1]] = fromPiece # to-piece becomes from-piece
    board[move[0][0]][move[0][1]] = '.' # from-piece becomes empty

    # Return new board layout
    return board   


## Part 3: Establish Chess Rules

This code block includes several functions to design the rules for movement of each piece on the board. This block will also contain the function to check if the current player is in check or check-mate. 

Note: These rules currently only contain the basic rules of chess; so advanced moves such as En Passant are not included for now.


In [3]:
# return True if the input move (from-square and to-square) is legal, else False
# this is the KEY function which contains the rules for each piece type 
def IsMoveLegal(board,player,fromSquare,toSquare):
    fromSquare_r = fromSquare[0]
    fromSquare_c = fromSquare[1]
    toSquare_r = toSquare[0]
    toSquare_c = toSquare[1]
    fromPiece = board[fromSquare_r][fromSquare_c]
    toPiece = board[toSquare_r][toSquare_c]

    if fromSquare == toSquare:
        return False

    
    # if the from-piece is a "king"
    if (fromPiece.lower() == 'k'):
        # calculate the col-diff = to-square-col - from-square-col
        col_diff = toSquare_c - fromSquare_c
        # calculate the row-diff = to-square-row - from-square-row
        row_diff = toSquare_r - fromSquare_r
        # if to-square is either empty or contains a piece that belongs to the enemy team
        if (toPiece == '.' or ((player == "black" and toPiece.isupper()) or (player == "white" and toPiece.islower()))):
            # return True for any of the following cases:
            if (
                # abs(col-diff) = 1 & abs(row_dif) = 0
                (abs(col_diff) == 1 and abs(row_diff) == 0) 
                # abs(col-diff) = 0 & abs(row_dif) = 1
                or (abs(col_diff) == 0 and abs(row_diff) == 1)  
                # abs(col-diff) = 1 & abs(row_dif) = 1
                or (abs(col_diff) == 1 and abs(row_diff) == 1)):
                    # return True
                    return True


    # else if the from-piece is a "rook"
    elif(fromPiece.lower() == 'r'):
        # if to-square is either in the same row or column as the from-square
        if (toSquare_r == fromSquare_r or toSquare_c == fromSquare_c):
            # if to-square is either empty or contains a piece that belongs to the enemy team
            if (toPiece == '.' or ((player == "black" and toPiece.isupper()) or (player == "white" and toPiece.islower()))):
                # if IsClearPath() - a clear path exists between from-square and to-square
                if IsClearPath(board,fromSquare,toSquare):
                    # return True
                    return True

    # else if the from-piece is a "bishop"
    elif(fromPiece.lower() == 'b'):
        # if to-square is diagonal wrt from-square
        if (abs(toSquare_r - fromSquare_r) == abs(toSquare_c - fromSquare_c)):
            # if to-square is either empty or contains a piece that belongs to the enemy team
            if (toPiece == '.' or ((player == "black" and toPiece.isupper()) or (player == "white" and toPiece.islower()))):
                # if IsClearPath() - a clear path exists between from-square and to-square
                if IsClearPath(board,fromSquare,toSquare):
                    # return True
                    return True


    # else if the from-piece is a "queen"
    elif(fromPiece.lower() == 'q'):
        ## SAME as "rook"
        # if to-square is either in the same row or column as the from-square
        if (toSquare_r == fromSquare_r or toSquare_c == fromSquare_c):
            # if to-square is either empty or contains a piece that belongs to the enemy team
            if (toPiece == '.' or ((player == "black" and toPiece.isupper()) or (player == "white" and toPiece.islower()))):
                # if IsClearPath() - a clear path exists between from-square and to-square
                if IsClearPath(board,fromSquare,toSquare):
                    # return True
                    return True
        ## SAME as "bishop"
        # if to-square is diagonal wrt from-square
        if (abs(toSquare_r - fromSquare_r) == abs(toSquare_c - fromSquare_c)):
            # if to-square is either empty or contains a piece that belongs to the enemy team
            if (toPiece == '.' or ((player == "black" and toPiece.isupper()) or (player == "white" and toPiece.islower()))):
                # if IsClearPath() - a clear path exists between from-square and to-square
                if IsClearPath(board,fromSquare,toSquare):
                    # return True
                    return True


    # else if the from-piece is a "knight"
    elif(fromPiece.lower() == 't'):
        # calculate the col-diff = to-square-col - from-square-col
        col_diff = toSquare_c - fromSquare_c
        # calculate the row-diff = to-square-row - from-square-row
        row_diff = toSquare_r - fromSquare_r
        # if to-square is either empty or contains a piece that belongs to the enemy team
        if (toPiece == '.' or ((player == "black" and toPiece.isupper()) or (player == "white" and toPiece.islower()))):
            # return True for any of the following cases:
            if (
                # col-diff = 1 & row_dif = -2
                (col_diff == 1 and row_diff == -2)
                # col-diff = 2 & row_dif = -1
                or (col_diff == 2 and row_diff == -1)
                # col-diff = 2 & row_dif = 1
                or (col_diff == 2 and row_diff == 1)
                # col-diff = 1 & row_dif = 2
                or (col_diff == 1 and row_diff == 2)
                # col-diff = -1 & row_dif = -2
                or (col_diff == -1 and row_diff == -2)
                # col-diff = -2 & row_dif = -1
                or (col_diff == -2 and row_diff == -1)
                # col-diff = -2 & row_dif = 1
                or (col_diff == -2 and row_diff == 1)
                # col-diff = -1 & row_dif = 2
                or (col_diff == -1 and row_diff == 2)):
                    # return True
                    return True


    # else if the from-piece is a "pawn"
    elif(fromPiece.lower() == 'p'):
        ## case - pawn wants to move one step forward (or backward if white)
        if (
            # if to-square is empty
            toPiece == '.' 
            # and is in the same column as the from-square
            and (toSquare_c == fromSquare_c)
            # and to-square is only + 1 or -1 row (with respect to player type)
            and ((player == "black" and toSquare_r == (fromSquare_r + 1)) 
            or (player == "white" and toSquare_r == (fromSquare_r - 1)))):
            # return True
            return True
        ## case - pawn can move two spaces forward (or backward if white) ONLY if pawn on starting row
        elif (
             # else if to-square is empty
             toPiece == '.' 
             # and is in same column as the from-square
             and (toSquare_c == fromSquare_c)
             # and from-square-row = 1 (or 6 if white) and to-square-row = from-square-row + 2 (or -2 if white)
             and ((player == "black" and fromSquare_r == 1 and toSquare_r == (fromSquare_r + 2)) 
             or (player == "white" and fromSquare_r == 6 and toSquare_r == (fromSquare_r - 2)))):
                # if IsClearPath() - a clear path exists between from-square and to-square
                if IsClearPath(board, fromSquare, toSquare):
                    # return True
                    return True
        ## case - pawn attacks the enemy piece if diagonal
        # else if there is piece diagonally forward  
        elif(
            # there is a piece at to-square (else if to-square is not empty)
            toPiece != '.'
            # and the to-square-column is +1 (right) or -1 (left) of from-square-column
            and (toSquare_c == (fromSquare_c - 1) or toSquare_c == (fromSquare_c + 1))
            # and that piece belongs to the enemy team (forward if black)
            and ((player == "black" and toPiece.isupper() and toSquare_r == (fromSquare_r + 1))
            # (or backward if white)
            or (player == "white" and toPiece.islower() and toSquare_r == (fromSquare_r - 1)))):
              # return True
              return True
    # if none of the other True's are hit above
    return False


# gets a list of legal moves for a given piece
# input = from-square
# output = list of to-square locations where the piece can move to
def GetListOfLegalMoves(board, player, fromSquare):
    # input is the current player and the given piece as the from-square
    # initialize the list of legal moves, i.e., to-square locations to []
    legal_moves = []
    # go through all squares on the board
    for row in range(8):
      for col in range(8):
          # for the selected square as to-square
          toSquare = (row,col)
          # call IsMoveLegal() with input as from-square and to-square and save the returned value
          legality = IsMoveLegal(board, player, fromSquare, toSquare)
          # if returned value is True
          if(legality == True):
              # call DoesMovePutPlayerInCheck() with input as from-square and to-square and save the returned value
              check = DoesMovePutPlayerInCheck(board, player, fromSquare, toSquare)
              # if returned value is False
              if (check == False):
                  # append this move (to-square) as a legal move
                  legal_moves.append(toSquare)
    # return the list of legal moves, i.e., to-square locations
    return legal_moves


# gets a list of all pieces for the current player that have legal moves
def GetPiecesWithLegalMoves(board, player):
    # initialize the list of pieces with legal moves to []
    legal_pieces = []
    # go through all squares on the board
    for row in range(8):
      for col in range(8):
          # for the selected square
          selected_square = (row,col)
          # if the square contains a piece that belongs to the current player's team
          if((player == "black" and board[row][col].islower()) or (player == "white" and board[row][col].isupper())):
              # call GetListOfLegalMoves() to get a list of all legal moves for the selected piece / square
              legal_moves = GetListOfLegalMoves(board, player, selected_square)
              # if there are any legel moves
              if(legal_moves):
                  # append this piece to the list of pieces with legal moves
                  legal_pieces.append(selected_square)
    # return the final list of pieces with legal moves
    return legal_pieces


# returns True if the current player is in checkmate, else False
def IsCheckmate(board, player):
    # call GetPiecesWithLegalMoves() to get all legal moves for the current player
    legal_pieces = GetPiecesWithLegalMoves(board, player)
    # if there is no piece with any valid move
    if not legal_pieces:
        # return True
        return True
    # else
    else:
        # return False
        return False


# returns True if the given player is in Check state
def IsInCheck(board, player):
    # determines enemy player based on the opposite distinction of the current player
    if player == "white":
      enemyPlayer = "black"
    elif player == "black":
      enemyPlayer = "white"
      
    # find given player's King's location = king-square
    king_square = (0,0)
    # go through all squares on the board
    for row in range(8):
      for col in range(8):
        # determines king-square for the current player
        if(player == "black" and board[row][col] == 'k'):
          king_square = (row,col)
        elif(player == "white" and board[row][col] == 'K'):
          king_square = (row,col)

    # go through all squares on the board
    for row in range(8):
      for col in range(8):
        selected_square = (row,col)
        toPiece = board[row][col]
        # if there is a piece at that location
        if(toPiece != '.' 
           # and that piece is of the enemy team
           and (player == "black" and toPiece.isupper()) or (player == "white" and toPiece.islower())):
            # call IsMoveLegal() for the enemy player from that square to the king-square
            legality = IsMoveLegal(board, enemyPlayer, selected_square, king_square)
            # if the value returned is True
            if(legality == True):
                # return True
                return True
            # else
            else:
                # do nothing and continue 
                continue
    # return False at the end
    return False


# helper function to figure out if a move is legal for straight-line moves (rooks, bishops, queens, pawns)
# returns True if the path is clear for a move (from-square and to-square), non-inclusive
def IsClearPath(board,fromSquare,toSquare):
    fromSquare_r = fromSquare[0]
    fromSquare_c = fromSquare[1]
    toSquare_r = toSquare[0]
    toSquare_c = toSquare[1]
    fromPiece = board[fromSquare_r][fromSquare_c]

    # if the from and to squares are only one square apart
    if abs(fromSquare_r - toSquare_r) <= 1 and abs(fromSquare_c - toSquare_c) <= 1:
        #The base case: just one square apart
        return True
    else:
        # if to-square is in the +ve vertical direction from from-square
        if toSquare_r > fromSquare_r and toSquare_c == fromSquare_c:
            # new-from-square = next square in the +ve vertical direction
            newSquare = (fromSquare_r + 1,fromSquare_c)
        # else if to-square is in the -ve vertical direction from from-square
        elif(toSquare_r < fromSquare_r and toSquare_c == fromSquare_c):
            # new-from-square = next square in the -ve vertical direction
            newSquare = (fromSquare_r - 1, fromSquare_c)
        # else if to-square is in the +ve horizontal direction from from-square
        elif(toSquare_r == fromSquare_r and toSquare_c > fromSquare_c):
            # new-from-square = next square in the +ve horizontal direction
            newSquare = (fromSquare_r, fromSquare_c + 1)
        # else if to-square is in the -ve horizontal direction from from-square
        elif(toSquare_r == fromSquare_r and toSquare_c < fromSquare_c):
            # new-from-square = next square in the -ve horizontal direction
            newSquare = (fromSquare_r, fromSquare_c - 1)
        # else if to-square is in the SE diagonal direction from from-square
        elif(toSquare_r > fromSquare_r and toSquare_c > fromSquare_c):
            # new-from-square = next square in the SE diagonal direction
            newSquare = (fromSquare_r + 1, fromSquare_c + 1)
        # else if to-square is in the SW diagonal direction from from-square
        elif(toSquare_r > fromSquare_r and toSquare_c < fromSquare_c):
            # new-from-square = next square in the SW diagonal direction
            newSquare = (fromSquare_r + 1, fromSquare_c - 1)
        # else if to-square is in the NE diagonal direction from from-square
        elif(toSquare_r < fromSquare_r and toSquare_c > fromSquare_c):
            # new-from-square = next square in the NE diagonal direction
            newSquare = (fromSquare_r - 1, fromSquare_c + 1)
        # else if to-square is in the NW diagonal direction from from-square
        elif(toSquare_r < fromSquare_r and toSquare_c < fromSquare_c):
            # new-from-square = next square in the NW diagonal direction
            newSquare = (fromSquare_r - 1, fromSquare_c - 1)

    # if new-from-square is not empty
    if(board[newSquare[0]][newSquare[1]] != '.'):
        # return False
        return False
    # else
    else:
        # return the result from the recursive call of IsClearPath() with the new-from-square and to-square
        result = IsClearPath(board, newSquare, toSquare)
        return result


# makes a hypothetical move (from-square and to-square)
# returns True if it puts current player into check
def DoesMovePutPlayerInCheck(board, player, fromSquare, toSquare):
    # given the move (from-square and to-square), find the 'from-piece' and 'to-piece'
    fromPiece = board[fromSquare[0]][fromSquare[1]]
    toPiece = board[toSquare[0]][toSquare[1]]
    # make the move temporarily by changing the 'board'
    board[toSquare[0]][toSquare[1]] = fromPiece
    board[fromSquare[0]][fromSquare[1]] = '.'
    # Call the IsInCheck() function to see if the 'player' is in check - save the returned value
    check_checker = IsInCheck(board, player)
    # Undo the temporary move
    board[toSquare[0]][toSquare[1]] = toPiece
    board[fromSquare[0]][fromSquare[1]] = fromPiece
    # return the value saved - True if it puts current player into check, False otherwise
    return check_checker
    

# Artificial Intelligence

In this section, you will write code for the Artificial Intelligence (AI) that will play a game of chess. You will write 2 types of AI:

1.   **RandomAI** - This part will contain code for moving a chess piece randomly.
2.   **MinMaxAI** - This part will contain code for moving a chess piece using the MinMax strategy discussed in the lecture.


## Part 4: Random AI

This function will perform a **random move** for the given player. The function will return the **move** (from-piece and to-piece), so long as the move is legal. 

In [4]:
def GetRandomMove(board, curPlayer):
    # Call GetPiecesWithLegalMoves function to receive list of the player's pieces with legal moves (if any)
    legal_pieces = GetPiecesWithLegalMoves(board, curPlayer)
    
    # Choose a piece from the legal_pieces list randomly
    fromPiece = random.choice(legal_pieces)

    # Call GetListOfLegalMoves function to receive list of legal moves for the randomly chosen piece
    legal_moves = GetListOfLegalMoves(board, curPlayer, fromPiece)
  
    # Choose a legal move from the legal_moves list randomly (acts as the to-piece)
    toPiece = random.choice(legal_moves)

    # Initialize random move based off of previous information (from-piece and to-piece)
    random_move = (fromPiece, toPiece)

    # TEST PRINT STATEMENTS to help keep track of the from-piece, potential moves, and to-piece
    # print('\nrandom piece:\t', board[fromPiece[0]][fromPiece[1]], ' at ', fromPiece)
    # print('legal moves:\t', legal_moves)
    # print(board[fromPiece[0]][fromPiece[1]], ' at ', fromPiece, ' moves to ', board[toPiece[0]][toPiece[1]], ' at ', toPiece)

    # Return the random move
    return random_move


## Part 5: MinMax AI

This code block includes functions below that will perform a move for the given player using the MinMax AI strategy. 

The **evl** function will evaluate the board if a move is performed, gives a 'score' for every piece, and uses that to calculate the score for the entire chess board. 

The second function performs the MinMax strategy, and returns the move (from-piece and to-piece). Currently, this is written in **2-ply (one Max and one Min)**.


In [5]:
def evl(board, player):
    # this function will calculate the score on the board, if a move is performed
    # give score for each of piece and calculate the score for the chess board
    
    # initialize evaluation score
    score = 0

    # go through all squares on the board 
    for row in range(8):
        for col in range(8):
          # point values for each piece referenced from chess.com***
          # find the given player's pawns: add +1 to score
          if(board[row][col].lower() == 'p'):
              # determines current player type to add score appropriately
              if(player == "white" and board[row][col].isupper()):
                  score = score + 1
              elif(player == "black" and board[row][col].islower()):
                  score = score + 1
          # find the given player's knights: add +3 to score
          elif(board[row][col].lower() == 't'):
              # determines current player type to add score appropriately
              if(player == "white" and board[row][col].isupper()):
                  score = score + 3
              elif(player == "black" and board[row][col].islower()):
                  score = score + 3
          # find the given player's bishops: add +3 to score
          elif(board[row][col].lower() == 'b'):
              # determines current player type to add score appropriately
              if(player == "white" and board[row][col].isupper()):
                  score = score + 3
              elif(player == "black" and board[row][col].islower()):
                  score = score + 3
          # find the given player's rooks: add +5 to score
          elif(board[row][col].lower() == 'r'):
              # determines current player type to add score appropriately
              if(player == "white" and board[row][col].isupper()):
                  score = score + 5
              elif(player == "black" and board[row][col].islower()):
                  score = score + 5
          # find the given player's queen: add +9 to score
          elif(board[row][col].lower() == 'q'):
              # determines current player type to add score appropriately
              if(player == "white" and board[row][col].isupper()):
                  score = score + 9
              elif(player == "black" and board[row][col].islower()):
                  score = score + 9
          # no score added for king, since king should not be captured

    # return evaluation score
    return score


def GetMinMaxMove(board, curPlayer, last_move):
    # return the best move for the current player using the MinMax strategy
    # to get the allocated points, searching should be 2-ply (one Max and one Min)

    # Following is the setup for a 2-ply game

    # determines enemy player based on the opposite distinction of the current player
    if(curPlayer == "white"):
      enemyPlayer = "black"
    elif(curPlayer == "black"):
      enemyPLayer = "white"

    # Initialize variables for best moves (from-piece, to-piece, res) and evaluation
    bestMove = ((),(), -sys.maxsize)
    bestEnemyMove = ((),(), sys.maxsize)
    res = 0


    # pieces = GetPiecesWithLegalMoves(curPlayer)
    pieces = GetPiecesWithLegalMoves(board, curPlayer)
    # for each piece in pieces
    for piece in pieces:
      # moves = GetListOfLegalMoves(curPlayer, piece)
      moves = GetListOfLegalMoves(board, curPlayer, piece)
      
      # for move in moves
      for move in moves:
          # recieves current player's from-piece and to-piece on the board
          fromPiece = board[piece[0]][piece[1]]
          toPiece = board[move[0]][move[1]]

          # perform the move temporarily
          board[move[0]][move[1]] = fromPiece
          board[piece[0]][piece[1]] = '.'

          # clears bestEnemyMove for next iteration
          bestEnemyMove = ((),(), sys.maxsize)
          
          # enemyPieces = GetPiecesWithLegalMoves(enemyPlayer)
          enemyPieces = GetPiecesWithLegalMoves(board, enemyPlayer)

          # for enemyPiece in enemyPieces
          for enemyPiece in enemyPieces:
              # enemyMoves = GetListOfLegalMoves(enemyPlayer, enemyPiece)
              enemyMoves = GetListOfLegalMoves(board, enemyPlayer, enemyPiece)

              # for enemyMove in enemyMoves
              for enemyMove in enemyMoves:

                  # recieves enemy from-piece and to-piece on the board
                  enemy_FromPiece = board[enemyPiece[0]][enemyPiece[1]]
                  enemy_ToPiece = board[enemyMove[0]][enemyMove[1]]

                  # perform the enemyMove temporarily
                  board[enemyMove[0]][enemyMove[1]] = enemy_FromPiece
                  board[enemyPiece[0]][enemyPiece[1]] = '.'

                  # res = evl(curPlayer)
                  res = evl(board, curPlayer)
          
                  # update the bestEnemyMove -- this is the MIN player trying to minimize from the 'res' evaluation values
                  if (res < bestEnemyMove[2]): # heuristics at bestEnemyMove[2] starts at sys +maximum
                      bestEnemyMove = (enemyPiece, enemyMove, res)
                  
                  # undo the enemyMove
                  board[enemyPiece[0]][enemyPiece[1]] = enemy_FromPiece
                  board[enemyMove[0]][enemyMove[1]] = enemy_ToPiece

          # update the bestMove -- this is the MAX player trying to maximize from the 'bestEnemyMove' evaluation values
          if (bestEnemyMove[2] > bestMove[2]): # heuristics at bestMove[2] starts at sys -minimum
            bestMove = (piece, move, bestEnemyMove[2])
           
          # undo the move
          board[piece[0]][piece[1]] = fromPiece
          board[move[0]][move[1]] = toPiece
          

    # if bestMove found without any doubt, pick that
    # proceeds if bestMove actually has values (neither from-piece or to-piece for the bestMove are empty)
    if(bestMove[0] != () and bestMove[1] != ()):
      # (from OPTIONAL recommendation) checks if the last move is not empty
      if last_move != ():
        # checks if current best move is the opposite of the last move made
        if last_move[0] == bestMove[1] and last_move[1] == bestMove[0]:

          # returns a random move to avoid repeated moves and decrease chances of stalemates
          # TEST PRINT STATEMENTS to help keep track of whether or not minmax had to resort to random moves instead
          # print('\nMinMaxAI DEFAULTS to RandomAI!')
          return GetRandomMove(board, curPlayer)

      # TEST PRINT STATEMENTS to help keep track of the from-piece, potential moves, and to-piece ###
      # print('\nminmax piece:\t', board[bestMove[0][0]][bestMove[0][1]], ' at ', bestMove[0])
      # legal_moves_list = GetListOfLegalMoves(board, curPlayer, bestMove[0])
      # print('legal moves:\t', legal_moves_list)
      # print(board[bestMove[0][0]][bestMove[0][1]], ' at ', bestMove[0], ' moves to ', board[bestMove[1][0]][bestMove[1][1]], ' at ', bestMove[1])

      # returns the bestMove for minmax
      return (bestMove[0], bestMove[1])

    # if bestMove not found, pick randomly
    else:
      # TEST PRINT STATEMENTS to help keep track of whether or not minmax had to resort to random moves instead
      # print('MinMaxAI DEFAULTS to RandomAI!')

      # calls on GetRandomMove to return a random move since a best move was not found
      return GetRandomMove(board, curPlayer)

    # OPTIONAL -- sometimes automated chess keeps on performing the moves again and again
    # e.g., move king left one square and then move king back - repeat
    # For this you will need to remember the previous move and see if the current best move is not the same and opposite as the previous move
    # If so, pick the second best move instead of the best move


# Gameplay
## Part 6: Game Setup & Main Loop

The code below is the game setup and main loop to start game-play between Random AI and MinMax AI. The game will end when either player wins, or when 100 turns have been reached (resulting in a draw).

I have commented out parts that would clear the board for each iteration, so each turn can be analyzed.

This block can be re-run several times to start new games. As seen from the general output, MinMax AI, being more sophisticated, typically wins each match; usually anywhere between 15-60 turns.

In [6]:
# initialize and setup the board
# player assignment and counter initializations
board = ChessBoardSetup()
player1Type = 'minmaxAI'
player1player = 'white'
player2Type = 'randomAI'
player2player = 'black'

currentPlayerIndex = 0
currentplayer = 'white'
turns = 0
N = 100
last_move = ()

DrawBoard(board)

# main game loop - while a player is not in checkmate or stalemate (<N turns)
while not IsCheckmate(board,currentplayer) and turns < N:
    #clear_output()
    #DrawBoard(board)

    # code to take turns and move the pieces
    if currentplayer == 'black':
        move = GetRandomMove(board, currentplayer)
        print("\nBLACK / Random AI plays!\n")
    else:
        turns = turns + 1
        move = GetMinMaxMove(board, currentplayer, last_move) # implemented OPTIONAL recommendation to avoid repeated moves
        last_move = move
        print("\nWHITE / MinMax AI plays!\n")
    
    board = MovePiece(board,move)
    currentPlayerIndex = (currentPlayerIndex+1)%2
    currentplayer = 'black' if currentplayer == 'white' else 'white'

    DrawBoard(board)
    time.sleep(.5)

# check and print - Stalemate or Checkmate
if(IsCheckmate(board,currentplayer)):
    print("CHECKMATE!")
    winnerIndex = (currentPlayerIndex+1)%2
    if(winnerIndex == 0):
        print("MinMax AI - WHITE - won the game in " + str(turns) + " turns!")
    else:
        print("Random AI - BLACK - won the game in " + str(turns) + " turns!")
else:
    print("STALEMATE!")
  

♖♘♗♕♔♗♘♖
♙♙♙♙♙♙♙♙
                
                
                
                
♟♟♟♟♟♟♟♟
♜♞♝♛♚♝♞♜

WHITE / MinMax AI plays!

♖♘♗♕♔♗♘♖
♙♙♙♙♙♙♙♙
                
                
♟              
                
  ♟♟♟♟♟♟♟
♜♞♝♛♚♝♞♜

BLACK / Random AI plays!

♖♘♗♕♔♗♘♖
♙♙♙  ♙♙♙♙
      ♙        
                
♟              
                
  ♟♟♟♟♟♟♟
♜♞♝♛♚♝♞♜

WHITE / MinMax AI plays!

♖♘♗♕♔♗♘♖
♙♙♙  ♙♙♙♙
      ♙        
♟              
                
                
  ♟♟♟♟♟♟♟
♜♞♝♛♚♝♞♜

BLACK / Random AI plays!

♖  ♗♕♔♗♘♖
♙♙♙♘♙♙♙♙
      ♙        
♟              
                
                
  ♟♟♟♟♟♟♟
♜♞♝♛♚♝♞♜

WHITE / MinMax AI plays!

♖  ♗♕♔♗♘♖
♙♙♙♘♙♙♙♙
      ♙        
♟              
  ♟            
                
    ♟♟♟♟♟♟
♜♞♝♛♚♝♞♜

BLACK / Random AI plays!

♖  ♗♕♔♗♘♖
♙♙♙♘♙♙  ♙
      ♙    ♙  
♟              
  ♟            
                
    ♟♟♟♟♟♟
♜♞♝♛♚♝♞♜

WHITE / MinMax AI plays!

♖  ♗♕♔♗♘♖
♙♙♙♘♙♙  ♙
      ♙    ♙  
♟♟            
                
                